In [ ]:
from SMT_solver import SMT_solver
import time
from z3 import *
import numpy as np
import matplotlib.pyplot as plt
import subprocess

plt.style.use('default')

import sys
sys.path.append("../common")
from utils import visualize, read_instance, save_solution, visualize_from_file, optimize


#reloads external modules when they are changed
%load_ext autoreload
%autoreload 2


# Try one instance

In [ ]:
ins_filename = "../instances/ins-25.txt"
W, n, rectangles = read_instance(ins_filename)
break_symmetries = True
allow_rotation = False
verbose = True

solver = 'z3'
logic = 'QF_IDL'
timeout = 30

print("width: ", W)
print("number of rectangles: ", len(rectangles))
print("largest rectangle:", np.argmax([r.w * r.h for r in rectangles])+1)
print("break_symmetries:", break_symmetries)
print("")

try:
    start = time.time()
    H, positioned_rectangles = optimize('SMT', W, rectangles, allow_rotation, verbose, break_symmetries, solver=solver, timeout = timeout, logic = logic)
    execution_time = time.time() - start
    print(f"\nExecution time in seconds: {execution_time:.3f}")
except subprocess.TimeoutExpired:
    print("\n----Timeout----\n")


In [ ]:
fig, ax = visualize(W, H, positioned_rectangles)
fig.suptitle(f"{ins_filename.split('/')[-1].split('.')[0]}, {n} rectangles, W = {W}, H = {H}")
fig.tight_layout(pad=1)
#plt.savefig("../" + ins_filename.split('.')[0])
plt.show()

# Compute All Instances

In [ ]:
solver = 'z3'
logic = 'QF_IDL'

save_to_file = False

output_folder = f"out/{logic}/"
input_folder = "../instances/"

plot = True
plot_folder = f"out/{logic}_plots/"

break_symmetries = True
allow_rotation = True

verbose = True

print("")
print(50*"=")
for i in range(40):
    ins_filename = input_folder + f"ins-{i+1}.txt"
    solution_filename = output_folder + f"ins-{i+1}-sol.txt"
    W, n, rectangles = read_instance(ins_filename)

    m = np.argmax([r.w * r.h for r in rectangles])
    print(f"\nInstance {i+1}")
    print("Width:", W)
    print("Number of rectangles:", len(rectangles))
    print("Largest rectangle measures:", (rectangles[m].w, rectangles[m].h))
    print("Largest rectangle index:", m+1)
    print("Breaking symmetries:", break_symmetries)
    print("")

    execution_time = 0.
    H = 0
    try:
        start = time.time()
        H, positioned_rectangles = optimize('SMT', W, rectangles, allow_rotation, verbose, break_symmetries, solver=solver, logic=logic)
        execution_time = time.time() - start
        print("\nFound height:", H)
        print(f"Execution time in seconds: {execution_time:.3f}")

        if save_to_file:
            save_solution(solution_filename, W, H, positioned_rectangles)
            with open(solution_filename, "a") as f:
                f.write(f"\nexecution time in seconds: {execution_time:.3f}")
        if plot:
            fig, ax = visualize(W, H, positioned_rectangles)
            fig.suptitle(
                f"{ins_filename.split('/')[-1].split('.')[0]}, {n} rectangles, W = {W}, H = {H}")
            fig.tight_layout(pad=1)
            if save_to_file:
                plt.savefig(plot_folder + ins_filename.split('/')[-1].split('.')[0])
            plt.show()

    except subprocess.TimeoutExpired:
        print("\n----Timeout----\n")

    print("")
    print(50*"=")
